# Mathematical programming
# Extremum and methods of its search

_Function_
$$f(x) = 2x_1^2 + 3x_2^2 + x_3^2 - x_1x_2 + \frac{x_1x_3}{2} + 10x_1$$

- *Examine the necessary and sufficient conditions for the existence of an extremum*
- *Search for minimum or maximum*
- *Gradient method with step splitting*
- *Fastest descent method (gradient descent method)*
- *Newton's method*
- *Conjugate gradient method*
- *Coordinate descent method*

In [22]:
import sympy
import visualize
import f_method

## Interactive function graph

In [23]:
visualize.graphic_plotly()

## Search for extremum (necessary and sufficient conditions)

In [24]:
# def variables
x1, x2, x3 = sympy.symbols('x1, x2, x3')
f = 2*x1**2+3*x2**2+x3**2-x1*x2+(x1*x3)/2+10*x1
# partial derivatives of the function
df_dx1 = sympy.diff(f, x1)
df_dx2 = sympy.diff(f, x2)
df_dx3 = sympy.diff(f, x3)

In [25]:
print('df dx1 = ', df_dx1)
print('df dx2 = ', df_dx2)
print('df dx3 = ', df_dx3)

df dx1 =  4*x1 - x2 + x3/2 + 10
df dx2 =  -x1 + 6*x2
df dx3 =  x1/2 + 2*x3


$ \frac{{df}}{{dx_1}} = 4x_1 - x_2 + \frac{{x_3}}{2} + 10 $

$ \frac{{df}}{{dx_2}} = -x_1 + 6x_2 $

$ \frac{{df}}{{dx_3}} = \frac{{x_1}}{2} + 2x_3 $


\begin{cases}
4x_1 - x_2 + \frac{x_3}{2} = -10 \\
-x_1 + 6x_2 = 0 \\
\frac{x_1}{2} + 2x_3 = 0
\end{cases}


In [26]:
# find solution of partial derivatives system
system_eq_solution = sympy.solve(
                                    (sympy.Eq(df_dx1, 0), sympy.Eq(df_dx2, 0), sympy.Eq(df_dx3, 0)), 
                                    (x1, x2, x3)
                                 )

In [27]:
print("System solution:")
print("x1 =", system_eq_solution[x1], '=', float(system_eq_solution[x1]))
print("x2 =", system_eq_solution[x2], '=', float(system_eq_solution[x2]))
print("x3 =", system_eq_solution[x3], '=', float(system_eq_solution[x3]))
print("f  =", f.subs({x1: system_eq_solution[x1], x2: system_eq_solution[x2], x3: system_eq_solution[x3]}),
      '=', float(f.subs({x1: system_eq_solution[x1], x2: system_eq_solution[x2], x3: system_eq_solution[x3]})))

System solution:
x1 = -240/89 = -2.696629213483146
x2 = -40/89 = -0.449438202247191
x3 = 60/89 = 0.6741573033707865
f  = -1200/89 = -13.48314606741573


_System solution:_
$$
\begin{align*}
x_1 &= -\frac{240}{89} \approx -2.6966 \\
x_2 &= -\frac{40}{89} \approx -0.4494 \\
x_3 &= \frac{60}{89} \approx 0.6742 \\
f  &= \frac{-1200}{89} \approx -13.4831 \\
\end{align*}
$$


_Hessian Matrix (H):_
$$H = 
\begin{bmatrix}
    \frac{{\partial^2 f}}{{\partial x_1^2}} & \frac{{\partial^2 f}}{{\partial x_1 \partial x_2}} & \frac{{\partial^2 f}}{{\partial x_1 \partial x_3}} \\
    \frac{{\partial^2 f}}{{\partial x_2 \partial x_1}} & \frac{{\partial^2 f}}{{\partial x_2^2}} & \frac{{\partial^2 f}}{{\partial x_2 \partial x_3}} \\
    \frac{{\partial^2 f}}{{\partial x_3 \partial x_1}} & \frac{{\partial^2 f}}{{\partial x_3 \partial x_2}} & \frac{{\partial^2 f}}{{\partial x_3^2}}
\end{bmatrix}
$$



In [28]:
# Calculating second partial derivatives
d2f_dx1_dx1 = sympy.diff(df_dx1, x1)
d2f_dx1_dx2 = sympy.diff(df_dx1, x2)
d2f_dx1_dx3 = sympy.diff(df_dx1, x3)

d2f_dx2_dx2 = sympy.diff(df_dx2, x2)
d2f_dx2_dx1 = sympy.diff(df_dx2, x1)
d2f_dx2_dx3 = sympy.diff(df_dx2, x3)

d2f_dx3_dx3 = sympy.diff(df_dx3, x3)
d2f_dx3_dx1 = sympy.diff(df_dx3, x1)
d2f_dx3_dx2 = sympy.diff(df_dx3, x2)

# Making a Hesse matrix
hessian_matrix = sympy.Matrix([
    [d2f_dx1_dx1, d2f_dx1_dx2, d2f_dx1_dx3],
    [d2f_dx2_dx1, d2f_dx2_dx2, d2f_dx2_dx3],
    [d2f_dx3_dx1, d2f_dx3_dx2, d2f_dx3_dx3]
])


In [29]:
print('Hessian matrix =', hessian_matrix)

Hessian matrix = Matrix([[4, -1, 1/2], [-1, 6, 0], [1/2, 0, 2]])


In [30]:
M1_diag = hessian_matrix[0:1, 0:1].det()
M2_diag = hessian_matrix[0:2, 0:2].det()
M3_diag = hessian_matrix.det()

_Minor 1 Diagonal_: $$M_1 = \begin{vmatrix} 4 \end{vmatrix} = 4$$

_Minor 2 Diagonal_: $$M_2 = \begin{vmatrix} 4 & -1 \\ -1 & 6 \end{vmatrix} = 23$$

_Minor 3 Diagonal_: $$M_3 = \begin{vmatrix} 4 & -1 & \frac{1}{2} \\ -1 & 6 & 0 \\ \frac{1}{2} & 0 & 2 \end{vmatrix} = \frac{89}{2}$$

_If all diagonal minors of the Hesse matrix are positive at the point then it indicates that the function has a local minimum at this point._

## Gradient method with step splitting

In [31]:
iterations, point = f_method.gradient_method_step_split(
                                                        f = f, 
                                                        x = [0, 0, 0], 
                                                        variables = (x1, x2, x3), 
                                                        alpha = 1.0 , 
                                                        beta = 0.5, 
                                                        epsilon = 0.001
                                                      )  
print(point)
print(f"The extremum is found in {iterations} iterations.\n")
print(f"x1 = {point[0]},\nx2 = {point[1]},\nx3 = {point[2]}\n")
print(f"The value of a function at an extremum:\nf = {f.subs({x1: point[0], x2: point[1], x3: point[2]})}")

[-2.69659113465144, -0.449527913537185, 0.674161030137189]
The extremum is found in 20 iterations.

x1 = -2.69659113465144,
x2 = -0.449527913537185,
x3 = 0.674161030137189

The value of a function at an extremum:
f = -13.4831460368704


## Fastest descent method

In [32]:
point, iterations = f_method.gradient_descent(f=f, 
                                  x=[0, 0, 0], 
                                  variables=(x1, x2, x3), 

                                 )
print(f"The extremum is found in {iterations} iterations.\n")
print(f"x1 = {point[0]},\nx2 = {point[1]},\nx3 = {point[2]}\n")
print(f"The value of a function at an extremum:\nf = {f.subs({x1: point[0], x2: point[1], x3: point[2]})}")


The extremum is found in 20 iterations.

x1 = -2.69662911251077,
x2 = -0.449437610043053,
x3 = 0.674156074035901

The value of a function at an extremum:
f = -13.4831460674133


## Newton's method

In [33]:
point, iterations = f_method.newton_method(f=f, 
                                  x=[0, 0, 0], 
                                  variables=(x1, x2, x3), 
                                  epsilon = 0.001
                                 )
print(f"The extremum is found in {iterations} iterations.\n")
print(f"x1 = {point[0]},\nx2 = {point[1]},\nx3 = {point[2]}\n")
print(f"The value of a function at an extremum:\nf = {f.subs({x1: point[0], x2: point[1], x3: point[2]})}")


The extremum is found in 2 iterations.

x1 = -2.696629213483146,
x2 = -0.449438202247191,
x3 = 0.6741573033707865

The value of a function at an extremum:
f = -13.4831460674157


## Conjugate gradient method

In [34]:
point, iterations = f_method.conjugate_gradient_method(f=f, 
                                              x=[0, 0, 0], 
                                              variables=(x1, x2, x3))
print(f"The extremum is found in {iterations} iterations.\n")
print(f"x1 = {point[0]},\nx2 = {point[1]},\nx3 = {point[2]}\n")
print(f"The value of a function at an extremum:\nf = {f.subs({x1: point[0], x2: point[1], x3: point[2]})}")

The extremum is found in 3 iterations.

x1 = -2.6966290622792304,
x2 = -0.4494382160841414,
x3 = 0.6741572994427261

The value of a function at an extremum:
f = -13.4831460674157


## Coordinate descent method

In [35]:
point, iterations = f_method.coordinate_descent_method(f=f, x0=[0, 0, 0], variables=(x1, x2, x3))

print(f"The extremum is found in {iterations} iterations.\n")
print(f"x1 = {point[0]},\nx2 = {point[1]},\nx3 = {point[2]}\n")
print(f"The value of a function at an extremum:\nf = {f.subs({x1: point[0], x2: point[1], x3: point[2]})}")

The extremum is found in 5 iterations.

x1 = -2.696623647241258,
x2 = -0.4494372619324507,
x3 = 0.6741559068779166

The value of a function at an extremum:
f = -13.4831460673583
